# TODO 

- Special keys treatment 
- user identification 
- improve logger (possibility to name the csv and to stop the app without restarting the kernel)
- 5 samples for each profile 
- Similarity measures (std, classification, nearest neighbor)

In [2]:
import pandas as pd 
import numpy as np
from datetime import datetime
from os import listdir
from os.path import isfile, join
import os
import math

# Data import from keylogger

In [3]:
#file_path = 'keylog.txt'
path_merged = "input_data_merged/"
path_preprocessed_merged = "preprocessed_input_data_merged/"
columns = ['Timestamp', 'Key', 'Action']
columns_bis = ['Timestamp', 'Key', 'Action', 'Shift']
#dataframe = pd.read_csv(file_path, sep=";", names=columns, encoding = "ISO-8859-1")
#display(dataframe)

# Preprocessing

In the event that a key is pressed for a 'long time' (at least 2 consecutive row with the same 'Key' value and 'Action' = press), we only keep the first pressed
<br>We also create some columns for specific keys like Shift, Alt, Win, Ctrl

In [4]:
path = 'input_data/'
user_list = files = [os.path.splitext(filename)[0] for filename in listdir(path)]
for i in range(0,len(user_list),5):
    all_filenames = []
    user_name = user_list[i].split('_')[0]
    for j in range(1, 6):    
        all_filenames = np.append(all_filenames, [path+user_name+'_'+str(j)+'.csv'])
    
    df_merged = (pd.read_csv(f, sep=';', names=columns, encoding = "ISO-8859-1") for f in all_filenames)
    df_merged = pd.concat(df_merged, ignore_index=False)
    df_merged.to_csv('input_data_merged/'+user_name+'.csv', sep=';', encoding = "ISO-8859-1")
    display(df_merged)

,Timestamp,Key,Action
0,"2021-11-03 08:44:53,007",Key.enter,release
1,"2021-11-03 08:44:57,927",Key.shift,press
2,"2021-11-03 08:44:58,035",'P',press
3,"2021-11-03 08:44:58,126",'P',release
4,"2021-11-03 08:44:58,180",Key.shift,release
...,...,...,...
981,"2021-11-03 08:55:42,870",'e',release
982,"2021-11-03 08:55:42,986",Key.shift,press
983,"2021-11-03 08:55:43,015",'.',press
984,"2021-11-03 08:55:43,124",'.',release


,Timestamp,Key,Action
0,"2021-11-10 09:57:34,370",Key.enter,release
1,"2021-11-10 09:57:38,291",Key.shift,press
2,"2021-11-10 09:57:38,518",'P',press
3,"2021-11-10 09:57:38,612",'P',release
4,"2021-11-10 09:57:38,642",Key.shift,release
...,...,...,...
1020,"2021-11-10 10:10:22,361",Key.shift,release
1021,"2021-11-10 10:10:22,606",Key.ctrl_l,press
1022,"2021-11-10 10:10:22,933",'\x13',press
1023,"2021-11-10 10:10:23,056",'\x13',release


,Timestamp,Key,Action
0,"2021-11-03 10:01:41,752",Key.enter,release
1,"2021-11-03 10:01:58,027",Key.shift,press
2,"2021-11-03 10:01:58,168",'P',press
3,"2021-11-03 10:01:58,302",'P',release
4,"2021-11-03 10:01:58,314",Key.shift,release
...,...,...,...
974,"2021-11-03 10:13:26,554",Key.backspace,release
975,"2021-11-03 10:13:26,632",Key.backspace,press
976,"2021-11-03 10:13:26,708",Key.backspace,release
977,"2021-11-03 10:13:27,223",'.',press


In [5]:
path_merged = "input_data_merged/"

def is_shifted_key(df, key_index):
    press = False
    release = False
    for i in range(key_index, df.index.start-1, -1):
        if df['Key'][i] == 'Key.shift' and df['Action'][i] == 'press':
            press = True
            for j in range(key_index, i, -1):
                if df['Key'][j] == 'Key.shift' and df['Action'][j] == 'release':
                    release = True
                    break
            break
    if press == True and release == False:
        return True
    else:
        return False

def preprocess_holding(dataframe):
    for i in range(dataframe.index.start, dataframe.index.stop):
        if i < dataframe.index.stop:
            current_key = dataframe['Key'][i]
            if dataframe['Action'][i] == 'press':
                press = True
                j = i + 1
                while press:
                    if j < dataframe.index.stop and dataframe['Action'][j] == 'press' and current_key == dataframe['Key'][j]:
                        dataframe = dataframe.drop([j])
                        dataframe = dataframe.reset_index(drop=True)
                    else:
                        break
    return dataframe
    
def preprocess_shift(dataframe):
    dataframe_bis = pd.DataFrame(columns=columns_bis)
    for i in range(dataframe.index.start, dataframe.index.stop):
        if is_shifted_key(dataframe, i):
            tmp = pd.DataFrame([[dataframe['Timestamp'][i], dataframe['Key'][i], dataframe['Action'][i], 1]], columns = columns_bis)
            dataframe_bis = dataframe_bis.append(tmp)
        else:
            tmp = pd.DataFrame([[dataframe['Timestamp'][i], dataframe['Key'][i], dataframe['Action'][i], 0]], columns = columns_bis)
            dataframe_bis = dataframe_bis.append(tmp)
    dataframe_bis = dataframe_bis.reset_index(drop=True)
    dataframe = dataframe_bis
    return dataframe

user_list = os.listdir(path_merged)
print(user_list)
for user in range(len(user_list)):
    dataframe = pd.read_csv(path_merged+user_list[user], sep=";", names=columns, encoding = "ISO-8859-1")
    dataframe = dataframe.reset_index(drop=True)
    dataframe = dataframe.drop([0])
    dataframe = dataframe.reset_index(drop=True)
    dataframe = preprocess_holding(dataframe)
    dataframe = preprocess_shift(dataframe)
    dataframe['Timestamp'] = pd.to_datetime(dataframe['Timestamp'], format='%Y-%m-%d %H:%M:%S,%f')  
    dataframe.to_csv(path_preprocessed_merged+user_list[user], sep=';', encoding = "ISO-8859-1")     
    display(dataframe)




['benjamin.csv', 'hugo.csv', 'johannes.csv']


,Timestamp,Key,Action,Shift
0,2021-11-03 08:44:53.007,Key.enter,release,0
1,2021-11-03 08:44:57.927,Key.shift,press,1
2,2021-11-03 08:44:58.035,'P',press,1
3,2021-11-03 08:44:58.126,'P',release,1
4,2021-11-03 08:44:58.180,Key.shift,release,0
...,...,...,...,...
4707,2021-11-03 08:55:42.870,'e',release,0
4708,2021-11-03 08:55:42.986,Key.shift,press,1
4709,2021-11-03 08:55:43.015,'.',press,1
4710,2021-11-03 08:55:43.124,'.',release,1


,Timestamp,Key,Action,Shift
0,2021-11-10 09:57:34.370,Key.enter,release,0
1,2021-11-10 09:57:38.291,Key.shift,press,1
2,2021-11-10 09:57:38.518,'P',press,1
3,2021-11-10 09:57:38.612,'P',release,1
4,2021-11-10 09:57:38.642,Key.shift,release,0
...,...,...,...,...
5112,2021-11-10 10:10:22.361,Key.shift,release,0
5113,2021-11-10 10:10:22.606,Key.ctrl_l,press,0
5114,2021-11-10 10:10:22.933,'\x13',press,0
5115,2021-11-10 10:10:23.056,'\x13',release,0


,Timestamp,Key,Action,Shift
0,2021-11-03 10:01:41.752,Key.enter,release,0
1,2021-11-03 10:01:58.027,Key.shift,press,1
2,2021-11-03 10:01:58.168,'P',press,1
3,2021-11-03 10:01:58.302,'P',release,1
4,2021-11-03 10:01:58.314,Key.shift,release,0
...,...,...,...,...
4878,2021-11-03 10:13:26.554,Key.backspace,release,0
4879,2021-11-03 10:13:26.632,Key.backspace,press,0
4880,2021-11-03 10:13:26.708,Key.backspace,release,0
4881,2021-11-03 10:13:27.223,'.',press,0


In [27]:
dataframe = pd.read_csv("test.csv", sep = ";", names=columns, encoding = "ISO-8859-1")
dataframe = preprocess_holding(dataframe)
dataframe = preprocess_shift(dataframe)
dataframe['Timestamp'] = pd.to_datetime(dataframe['Timestamp'], format='%Y-%m-%d %H:%M:%S,%f')  
dataframe.to_csv("test_preprocessed.csv", sep=';', encoding = "ISO-8859-1")     
display(dataframe)

,Timestamp,Key,Action,Shift
0,2021-11-10 23:49:46.236,Key.enter,release,0
1,2021-11-10 23:49:50.144,Key.shift,press,1
2,2021-11-10 23:49:50.299,'L',press,1
3,2021-11-10 23:49:50.397,Key.shift,release,0
4,2021-11-10 23:49:50.401,'l',release,0
...,...,...,...,...
662,2021-11-10 23:50:58.821,'s',release,0
663,2021-11-10 23:50:58.947,Key.shift,press,1
664,2021-11-10 23:50:59.041,'.',press,1
665,2021-11-10 23:50:59.128,'.',release,1


## Reference Base Building

**columns_nd** = *nd for normalize data* <br>
**Hold Duration (HD)** = *Duration between press and release of the same key* <br>
**Hold time in digraph (HTD)** = *Dwell time with the context of the preceding keystroke* <br/>
**Cumulative Dwell Time (CDT)** = *Holding time for two consecutive keys without flying time* <br>
**Digraph latency (DL)** = *Flight time between two keystrokes*





In [21]:

columns_nd = ['Key1', 'Key2', 'HD', 'HTD', 'CDT', 'DL', 'Shift']# 'Shift', 'Alt', 'Ctrl', 'Win']
df = pd.DataFrame(columns = columns_nd)
df_bis = pd.DataFrame(columns = columns_nd)
columns_ref = ["Key", "Shift", "HD_mean", "HD_std", "DL_mean", "DL_std"]
path_reference = "reference_1/"
date_time_str = '2021-11-03 10:13:00.358'

date_time_obj = datetime. strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')

def search_for_press(df):
    for i in range(df.index.start, df.index.stop):
        if df['Action'][i] == 'press':
            return i

def search_for_release(df, press_index):
    for i in range (press_index, df.index.stop):
        if (df['Key'][i] == df['Key'][press_index] or df['Key'][i].lower() == df['Key'][press_index].lower()) and df['Action'][i] == 'release':
            return i
        
def search_for_following_key(df, key_index):
    for i in range(key_index, df.index.stop):
        if df['Key'][i] != df['Key'][key_index] and df['Action'][i] == 'press':
            return i
def metrics_computation(dataframe):
    df = pd.DataFrame(columns = columns_nd)                  
    while dataframe.empty == False:
        end = False
        if dataframe['Action'][0] == 'release':
            dataframe = dataframe.drop([0])
            dataframe = dataframe.reset_index(drop=True)

        press_index = search_for_press(dataframe)
        release_index = search_for_release(dataframe, press_index)
        
        
        
        if dataframe.index.stop-dataframe.index.start <= 2:
            end = True
        if end == False:
            following_key_press_index = search_for_following_key(dataframe, press_index)
            following_key_release_index = search_for_release(dataframe, following_key_press_index)
            Key1 = dataframe['Key'][press_index]
            Key2 = dataframe['Key'][following_key_press_index]

            HD = dataframe['Timestamp'][release_index] - dataframe['Timestamp'][press_index]
            HD = HD.total_seconds()
            HTD = dataframe['Timestamp'][following_key_release_index] - dataframe['Timestamp'][following_key_press_index]
            HTD = HTD.total_seconds()
            CDT = HD + HTD
            DL = dataframe['Timestamp'][following_key_press_index] - dataframe['Timestamp'][release_index]
            DL = DL.total_seconds()
            shift = dataframe['Shift'][press_index]
            dataframe = dataframe.drop([press_index, release_index])
            dataframe = dataframe.reset_index(drop=True)
            tmp = pd.DataFrame([[Key1, Key2, HD, HTD, CDT, DL, shift]], columns = columns_nd)
            df = df.append(tmp)
        else:
            Key1 = dataframe['Key'][press_index]
            HD = dataframe['Timestamp'][release_index] - dataframe['Timestamp'][press_index]
            HD = HD.total_seconds()
            HTD = None
            CDT = None
            DL = None
            shift = dataframe['Shift'][press_index]
            dataframe = dataframe.drop([press_index, release_index])
            tmp = pd.DataFrame([[Key1, None, HD, HTD, CDT, DL, shift]], columns = columns_nd)
            df = df.append(tmp)
    df = df.reset_index(drop=True)
    return df

def contains(dataframe, elem):
    for i in dataframe.index:
        if elem == dataframe['Key'][i]:
            return True

def compute_reference(dataframe):
    df_ref = pd.DataFrame(columns = columns_ref)
    for i in dataframe.index:
        var = dataframe['Key1'][i]
        if contains(df_ref, var):
            continue
        hold_durations = []
        flight_durations = []
        hold_durations = np.append(hold_durations, dataframe['HD'][i])
        if dataframe['DL'][i] != None:
                        flight_durations = np.append(flight_durations, dataframe['DL'][i])
        for j in range(i+1, dataframe.index.stop):
            if dataframe['Key1'][i] == dataframe['Key1'][j] and dataframe['Shift'][j] == dataframe['Shift'][i]:
                    hold_durations = np.append(hold_durations, dataframe['HD'][j])
                    if dataframe['DL'][j] != None:
                        flight_durations = np.append(flight_durations, dataframe['DL'][j])
        mean_HD = np.mean(hold_durations)
        std_HD = np.std(hold_durations)
        mean_DL = np.mean(flight_durations)
        std_DL = np.std(flight_durations)
        result = pd.DataFrame([[dataframe['Key1'][i], dataframe['Shift'][i], mean_HD, std_HD, mean_DL, std_DL]], columns = columns_ref)
        df_ref = df_ref.append(result)    
        df_ref = df_ref.reset_index(drop=True)
                
    """
    while dataframe.empty == False:
        hold_durations = []
        flight_durations = []
        key_save = dataframe['Key1'][0]
        shift_save = dataframe["Shift"][0]
        for i in dataframe.index:
            if i < dataframe.index.stop:
                if dataframe['Key1'][i] == key_save and shift_save == dataframe['Shift'][i]:
                    hold_durations = np.append(hold_durations, dataframe['HD'][i])
                    if i != dataframe.index.stop - 1:
                        flight_durations = np.append(flight_durations, dataframe['DL'][i])
                    dataframe = dataframe.drop([i])
                    dataframe = dataframe.reset_index(drop=True)       
        mean_HD = np.mean(hold_durations)
        std_HD = np.std(hold_durations)
        mean_DL = np.mean(hold_durations)
        std_DL = np.std(flight_durations)
        result = pd.DataFrame([[key_save, shift_save, mean_HD, std_HD, mean_DL, std_DL]], columns = columns_ref)
        df_ref = df_ref.append(result)
    """
    return df_ref


        

user_list = os.listdir(path_preprocessed_merged)       
print(user_list)
for user in range(len(user_list)):
    dataframe = pd.read_csv(path_preprocessed_merged+user_list[user], sep=";", names=columns_bis, encoding = "ISO-8859-1")
    dataframe = dataframe.reset_index(drop=True)
    dataframe = dataframe.drop([0])
    dataframe = dataframe.reset_index(drop=True)
    dataframe['Timestamp'] = pd.to_datetime(dataframe['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')   
    df_bis = metrics_computation(dataframe)
    df_ref = compute_reference(df_bis)
    df_ref = df_ref.reset_index(drop=True)
    df_ref.to_csv(path_reference+user_list[user], sep=';', encoding = "ISO-8859-1")     
    display(df_ref)
    
       





['benjamin.csv', 'hugo.csv', 'johannes.csv']


,Key,Shift,HD_mean,HD_std,DL_mean,DL_std
0,Key.shift,1,0.279894,0.166254,-0.156221,0.094029
1,'P',1,0.087000,0.022935,0.207800,0.145478
2,'o',0,0.093805,0.013640,0.101286,0.162329
3,'r',0,0.094156,0.018149,0.071714,0.129537
4,'t',0,0.095900,0.010504,0.034650,0.054827
5,'e',0,0.104109,0.025709,0.054751,0.153366
6,'z',0,0.123390,0.027449,0.025390,0.118388
7,Key.space,0,0.094949,0.012327,0.101423,0.154737
8,'c',0,0.103545,0.013157,0.047121,0.067123
9,'v',0,0.093933,0.017349,0.072567,0.086484


C:\Users\hugod\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\hugod\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\hugod\anaconda3\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\hugod\anaconda3\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
C:\Users\hugod\anaconda3\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,Key,Shift,HD_mean,HD_std,DL_mean,DL_std
0,Key.shift,1,0.348316,0.219469,-0.134504,0.220269
1,'P',1,0.091400,0.018084,0.113600,0.035752
2,'o',0,0.091808,0.027584,0.101658,0.173008
3,'r',0,0.095229,0.027034,0.056373,0.111068
4,'t',0,0.082236,0.018584,0.102691,0.181403
5,'e',0,0.089922,0.027776,0.069931,0.165775
6,'z',0,0.104233,0.032770,0.120558,0.212683
7,Key.space,0,0.095668,0.019140,0.234318,0.305708
8,'c',0,0.102862,0.024284,0.027379,0.077295
9,'v',0,0.109833,0.025839,-0.007533,0.037009


KeyError: None

In [28]:
dataframe = pd.read_csv("test_preprocessed.csv", sep=";", names=columns_bis, encoding = "ISO-8859-1")
dataframe = dataframe.reset_index(drop=True)
dataframe = dataframe.drop([0])
dataframe = dataframe.reset_index(drop=True)
display(dataframe)
dataframe['Timestamp'] = pd.to_datetime(dataframe['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')   
df_bis = metrics_computation(dataframe)
display(df_bis)
df_ref = compute_reference(df_bis)
df_ref = df_ref.reset_index(drop=True)
display(df_ref)
df_ref.to_csv("test_reference.csv", sep=';', encoding = "ISO-8859-1") 

,Timestamp,Key,Action,Shift
0,2021-11-10 23:49:46.236,Key.enter,release,0
1,2021-11-10 23:49:50.144,Key.shift,press,1
2,2021-11-10 23:49:50.299,'L',press,1
3,2021-11-10 23:49:50.397,Key.shift,release,0
4,2021-11-10 23:49:50.401,'l',release,0
...,...,...,...,...
662,2021-11-10 23:50:58.821,'s',release,0
663,2021-11-10 23:50:58.947,Key.shift,press,1
664,2021-11-10 23:50:59.041,'.',press,1
665,2021-11-10 23:50:59.128,'.',release,1


,Key1,Key2,HD,HTD,CDT,DL,Shift
0,Key.shift,'L',0.253,0.102,0.355,-0.098,1
1,'L','a',0.102,0.091,0.193,0.074,1
2,'a',Key.space,0.091,0.124,0.215,0.202,0
3,Key.space,'p',0.124,0.123,0.247,0.001,0
4,'p','r',0.123,0.084,0.207,0.002,0
...,...,...,...,...,...,...,...
328,'u','r',0.063,0.091,0.154,0.028,0
329,'r','s',0.091,0.118,0.209,0.002,0
330,'s',Key.shift,0.118,0.271,0.389,0.126,0
331,Key.shift,'.',0.271,0.087,0.358,-0.177,1


,Key,Shift,HD_mean,HD_std,DL_mean,DL_std
0,Key.shift,1,0.260300,0.058749,-0.132000,0.055519
1,'L',1,0.102000,0.000000,0.074000,0.000000
2,'a',0,0.096000,0.020359,0.045750,0.081958
3,Key.space,0,0.095059,0.018580,0.282451,0.362860
4,'p',0,0.117000,0.022486,-0.024600,0.024013
5,'r',0,0.105000,0.040216,0.012882,0.081272
6,'o',0,0.123900,0.043052,-0.011200,0.072360
7,'d',0,0.113556,0.015840,-0.009556,0.061972
8,'t',0,0.091900,0.027061,0.072050,0.163429
9,Key.backspace,0,0.092192,0.021904,0.332615,0.316423


# Scaled Manhattan Verifier

In [29]:
path = 'reference_1/'
user_list = files = [os.path.splitext(filename)[0] for filename in listdir(path)]
print(user_list)
def scaled_manhattan_verifier(vector):
    distance_to_users = np.array([[0,0]])
    distance_to_users = np.delete(distance_to_users, 0, 0)
    index = 0

    for user in range(len(user_list)):
        user_ref = pd.read_csv(path+user_list[user]+".csv", sep=";", names=columns_ref, encoding = "ISO-8859-1")
        user_ref = user_ref.reset_index(drop=True)
        user_ref = user_ref.drop([0])
        user_ref = user_ref.reset_index(drop=True)
        distance = 0
        for i in range(vector.shape[0]):
            for j in range(user_ref.shape[0]):
                if user_ref['Key'][j] == vector['Key'][i]:
                   if float(user_ref['HD_std'][j]) != 0 and math.isnan(float(user_ref['HD_std'][j])) == False:
                    distance += abs(float(vector['HD_mean'][i]) - float(user_ref['HD_mean'][j]))/float(user_ref['HD_std'][j])
                   if float(user_ref['DL_std'][j]) != 0 and math.isnan(float(user_ref['DL_std'][j])) == False:
                    distance += abs(float(vector['DL_mean'][i]) - float(user_ref['DL_mean'][j]))/float(user_ref['DL_std'][j])
        distance_to_users = np.append(distance_to_users, [[index, distance]], axis = 0)
        index += 1
        distance = 0
    minim = float('inf')
    res = 0
    display(distance_to_users)
    for i in range(len(distance_to_users)):
        if distance_to_users[i][1] < minim:
            minim = distance_to_users[i][1]
            res = i
    print(res)
    print('Vous êtes '+user_list[res])
                    
vector = pd.read_csv("test_reference.csv", sep=";", names=columns_ref, encoding = "ISO-8859-1")
vector = vector.reset_index(drop=True)
vector = vector.drop([0])
vector = vector.reset_index(drop=True)
display(vector)
scaled_manhattan_verifier(vector)     
            
            

['benjamin', 'hugo']


,Key,Shift,HD_mean,HD_std,DL_mean,DL_std
0,Key.shift,1,0.2603,0.05874870211332333,-0.132,0.055519365990616285
1,'L',1,0.102,0.0,0.074,0.0
2,'a',0,0.096,0.020359273071502332,0.04575,0.08195844983892754
3,Key.space,0,0.09505882352941177,0.01858041533201114,0.2824509803921569,0.36286006127385567
4,'p',0,0.11699999999999999,0.022485550916088313,-0.0246,0.024013329631685815
5,'r',0,0.105,0.04021559545309579,0.01288235294117647,0.0812721516250049
6,'o',0,0.12389999999999998,0.043052177645271326,-0.0112,0.0723599336649779
7,'d',0,0.11355555555555553,0.015840251899730432,-0.009555555555555555,0.061971518746941075
8,'t',0,0.09190000000000001,0.02706085734044655,0.07205,0.16342872299568395
9,Key.backspace,0,0.09219230769230768,0.021903668872112994,0.33261538461538465,0.3164234111881246


array([[ 0.        , 75.68421454],
       [ 1.        , 60.59888118]])

1
Vous êtes hugo
